In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import math as m
from matplotlib.lines import Line2D

df = pd.read_csv(r'/content/sample_data/Cleantech Solar.csv')
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format = True) # The date-time format is set to true for faster parsing inside the dataset

fig, ax = plt.subplots(figsize = (12, 8)) # The figsize parameter is used to specify the width and height of the figure in inches. In this case, the figure size is set to 12 inches by 8 inches.

conditions = [(df['GHI'] < 2),
              (df['GHI'] >= 2) & (df['GHI'] < 4),
              (df['GHI'] >= 4) & (df['GHI'] < 6),
              (df['GHI'] >= 6)]
colors = ['navy', 'cyan', 'orange', 'brown']
ax.scatter(df['Date'], df['PR'], c = np.select(conditions, colors), s = 5) # All the scatter points are plotted inside the graph and their individual colors are set according to GHI

rolling_mean = df['PR'].rolling(window = 30).mean()
a, = ax.plot(df['Date'], rolling_mean, color = 'red', linewidth = 2.2, label = '30-d moving average of PR') # Plot for the 30-d moving average of PR

ax.set_title('Performance Ratio Evolution (From 2019-07-01 to 2022-03-24)')
ax.set_xlabel('Date')
ax.set_ylabel('Performance Ratio [%]')

ax.set_ylim(ymin = 0)
ax.set_ylim(ymax = 100)
ax.set_yticks(range(0, 110, 10))


In [ ]:
# All the calculations are done for plotting the Target Budget Yield Performance Ratio. First the a list of budget data is calculated and then it is mapped with all the dates with the PR column.
start_date = pd.to_datetime('2019-07-01')
end_date = pd.to_datetime('2022-03-24')
num_years = m.ceil((end_date.year - start_date.year) + ((end_date.month - start_date.month) / 12))
budget_list = [73.9 * (1 - 0.008 * i) for i in range(num_years)]

# The Date column is converted to a list to map with the budget_list elements
df['Date'] = pd.to_datetime(df['Date'])
l = df['Date'].tolist()

# All the Dates are passed inside the condition for it to be mapped with the budget_list after every one passing year.
labels = []
for date in l:
    if date < datetime(2020, 7, 1):
        labels.append(budget_list[0])
    elif date < datetime(2021, 7, 1):
        labels.append(budget_list[1])
    else:
        labels.append(budget_list[2])

In [ ]:
b, = ax.plot(l, labels, color = 'darkgreen', linewidth = 1.5, label = 'Target Budget Yield Performance Ratio [1Y-73.9%, 2Y-73.3%, 3Y-72.7%]') # Final Plot for the Target Budget

# For calculating the total no. of scatter points that are above the target budget performance ratio
scatter_points = df['PR']
above_budget = np.where(scatter_points > labels[:len(df.index)])
Above_budget = len(above_budget[0])
c, = ax.plot([], linestyle = 'None', label = f'Points above target budget PR = {Above_budget}/{len(df)} = {round(Above_budget/len(df) * 100, 1)}%')

ax.set_xlim([df['Date'].min(), pd.to_datetime('24/03/2022')]) # For setting the max

In [ ]:
legend1 = ax.legend(handles=[
    Line2D([], [], linestyle = 'None'),
    Line2D([], [], color = 'navy', marker = 'o', linestyle = 'None', markersize = 5),
    Line2D([], [], color = 'cyan', marker = 'o', linestyle = 'None', markersize = 5),
    Line2D([], [], color = 'orange', marker = 'o', linestyle = 'None', markersize = 5),
    Line2D([], [], color = 'brown', marker = 'o', linestyle = 'None', markersize = 5)
], loc = 'upper center', labels = ['Daily Irradiation [KWh/m2]','< 2', '2 ~ 4', '4 ~ 6', '> 6'], ncol = 5, fontsize = 9)



legend2 = ax.legend(handles = [b, a, c], loc = 'center', ncol = 1, fontsize = 9)

day7 = round(sum(df['PR'][:7].tolist()) / 7, 1)
day30 = round(sum(df['PR'][:30].tolist()) / 30, 1)
day60 = round(sum(df['PR'][:60].tolist()) / 60, 1)
day90 = round(sum(df['PR'][:90].tolist()) / 90, 1)
day365 = round(sum(df['PR'][:365].tolist()) / 365, 1)
life_time = round(sum(df['PR'].tolist()) / len(df['PR']), 1)

legend3 = ax.legend(handles=[
    Line2D([], [], linestyle = 'None', label = f'Average PR last 7-d: {day7}%'),
    Line2D([], [], linestyle = 'None', label = f'Average PR last 30-d: {day30}%'),
    Line2D([], [], linestyle = 'None', label = f'Average PR last 60-d: {day60}%'),
    Line2D([], [], linestyle = 'None', label = f'Average PR last 90-d: {day90}%'),
    Line2D([], [], linestyle = 'None', label = f'Average PR last 365-d: {day365}%'),
    Line2D([], [], linestyle = 'None', label = f'Average PR Life Time: {life_time}%')
], loc = 'lower right', fontsize = 9)

In [ ]:
ax.add_artist(legend1)
ax.add_artist(legend2)
ax.add_artist(legend3)

ax.grid(True)

plt.subplots_adjust(hspace = 0.5)

plt.show()